# Welcome to the AIShield Integration Notebook!

**We are delighted to have you here!**

## What to Expect !!
This notebook serves as a comprehensive demonstration of the seamless integration capabilities of AIShield. As a powerful tool for securing your AI/ML assets against adversarial threats, AIShield ensures the integrity and robustness of your models, preventing financial loss, protecting brand reputation, and mitigating the risk of intellectual property theft.

## What to Do !!
- **Explore the notebook** at your own pace to gain deep insights into the functionalities and features offered by AIShield.
- **Run the first code cell/block**, which includes **Support functionalities** crucial for the smooth execution of the notebook. This setup will enhance your experience and enable the seamless integration of AIShield.
- Even without running the entire notebook, you can get a glimpse of AIShield's capabilities and its approach to vulnerability assessments through sample scenarios.
- **Run the code cells** to witness firsthand how AIShield seamlessly integrates with your existing codebase, empowering you to protect your AI models against adversarial attacks.
- **Experiment and adapt the code** to suit your specific use cases, leveraging the flexibility and customization options provided by AIShield.
- Should you have any questions or require assistance, our dedicated [AIShield.Contact@bosch.com](mailto:AIShield.Contact@bosch.com) is just a click away.

## What You'll Get !!
Throughout this notebook, you'll find:
- Code snippets demonstrating step-by-step procedures for incorporating AIShield into your existing workflows, ensuring comprehensive protection against adversarial threats.
- Detailed guidance on the integration process, including any additional inputs required for thorough vulnerability analysis.
- A comprehensive threat-informed defense model, empowering you to understand the vulnerabilities in your AI models and undertake appropriate remediation measures.
- Report artifacts and a sample attack dataset, providing valuable insights into potential attack scenarios and assisting in devising robust defense strategies.
- In-depth explanations of defense remediation techniques, equipping you with the knowledge and tools to safeguard your AI assets effectively.

## <span style="color:teal">AIShield Website and LinkedIn</span>
To learn more about AIShield and its cutting-edge features, visit the official [AIShield website](https://www.boschaishield.com/). Connect with AIShield on [LinkedIn](https://www.linkedin.com/company/bosch-aishield/about/) to stay updated on the latest advancements in AI security.

We hope this notebook empowers you to seamlessly integrate AIShield into your projects, ensuring the utmost protection for your AI/ML assets. If you have any questions or need further assistance, please don't hesitate to reach out. **Happy exploring!**


<a target="_blank" href="https://colab.research.google.com/github/bosch-aisecurity-aishield/Reference-Implementations/blob/main/Product_Taskpair_wise/Text_Classification/Text_Classification_Extraction.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## 1.0 Support Functionalities:

In [1]:
def make_code_input_scrollable():
    """Description : To make the input code cell scrollable for jupyternotebook"""
    from IPython.display import display, HTML
    display(HTML('''
        <style>
        .scrollable-code-input {
            max-height: 500px;
            overflow-y: scroll;
        }
        </style>
        <script>
        var codeInputCells = document.querySelectorAll('div.code_cell .input');
        codeInputCells.forEach(function(cell) {
            cell.classList.add('scrollable-code-input');
        });
        </script>
    '''))

make_code_input_scrollable()

def display_pdf(runtime_environment,output_filename, pdf_path=None, file_id=None):

    """Descrption : To embeded pdf file in the ntoebook"""

    from IPython.display import display, HTML
    import os
    output_file = output_filename+'.html'

    # Check if the output file exists
    if os.path.exists(output_file):
        with open(output_file, 'r') as f:
            saved_output = f.read()
            display(HTML(saved_output))
    else:
        if 'colab' in runtime_environment.lower():
            # Generate the Google Drive Viewer URL
            if file_id is None:
                print(pdf_path)
                # Generate the Google Drive Viewer URL
                drive_viewer_url = f'https://drive.google.com/viewerng/viewer?embedded=true&url={pdf_path}'
                html_code = f'<iframe src="{drive_viewer_url}" width="100%" height="600px"></iframe>'
            else:
                viewer_url = f"https://drive.google.com/file/d/{file_id}/preview"
                html_code = f'<iframe src="{viewer_url}" width="100%" height="600px"></iframe>'
        else:
            html_code = f'<embed src="{pdf_path}" width="100%" height="600px" type="application/pdf">'
        # Save the output to a file
        with open(output_file, 'w') as f:
            f.write(html_code)

        # Display the output
        display(HTML(html_code))


def check_runtime_environment():
    """
    Description : Check notebook is running on colab or jupyter-notebook
    """
    import os
    colab = False
    if 'google.colab' in str(get_ipython()):
        colab = True
        return 'Colab' , colab
    else:
        return 'Jupyter Notebook' , colab

# Check the runtime environment
runtime_environment,colab_running  = check_runtime_environment()

# print("Notebook is running in", runtime_environment)





<img src="https://aisdocs.blob.core.windows.net/reference/Workflow Images/wk1.png"/>

## 2.0 Below code cell/block performs Step 1 to 4 with following these sequential steps :
**STEP 1 : Installing libraries:** The required libraries will be installed to provide necessary functionalities for the task at hand.

**STEP 2 : Loading the dataset:** The dataset containing the relevant information will be loaded into the program, allowing access to the data for analysis and modeling.

**STEP 3 : Model development and training:** A sophisticated model will be developed to analyze and interpret the dataset. The model will then be trained using appropriate techniques to optimize its performance.

**STEP 4 : Preparing the Data, Model, and Label:** Prior to further analysis, essential steps such as data preprocessing, model configuration, and label preparation will be carried out. This will ensure that the data, model, and labels are properly structured and ready for subsequent stages needed for aishield.



In [2]:
#@title Show/Hide Code
'''
Description: user input about dataset information, for eg: number of classes, input_shape
'''

num_classes=2
model_encryption=1 # Text Classification needs model should be in encripted format
run_code = True
install = True #to install the prerequisite libraries or not
input_shape = 100 #length of the dataset , number of features or number of cloumns-1

## installing libraries if not and then importing
if install:
    print("Started of installing all the prerequisites packages")
    !pip install pandas==1.1.5
    !pip install numpy==1.22
    !pip install requests==2.28.0
    !pip install humanfriendly==9.2

    print("Starting installing aishield library")
    !pip install aishield
    print("Installed all the prerequisites packages")

else:
    print("Skipped the installation of all the prerequisites packages")


import io
import os
import copy
import json
import time
import shutil
import requests
import zipfile
import requests
import numpy as np
import pandas as pd

import aishield as ais



def download_artifacts(url):
    """
    Description :
    This function will download the dataset from the given file_path
    parameter:

        input:
        file_path : url of the dataset from where need to download the dataset

        output: None
    """
    file_path = os.path.basename(url)

    # Download the file
    response = requests.get(url)
    with open(file_path, 'wb') as f:
        f.write(response.content)

#extract content from the nested zip file
def extract_zip(zip_file_path, extract_to_folder):

    """
    Description :
    This function will extract the zipped file to a given destination folder

        input:
        zip_file_path : download zipped file path
        extract_to_folder: destination folder to extract the content of zipped files

        output: None
    """
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all contents into the target folder
        zip_ref.extractall(extract_to_folder)
    zip_ref.close()



def download_extract_artifacts(download_url, zipped_file_path, destination_path):

    """
    Description :
    This function will trigger the download and extraction function one by one

        input:
        download_url: url from where content need to be downloaded
        zip_file_path : download zipped file path
        destination_path: destination folder to extract the content of zipped files

        output: None
    """

    # download the artifacts from the given url
    try:
        download_artifacts(download_url)
        # extract the zipped artifacts in to given destination folder
        extract_zip(zipped_file_path, destination_path)

    except Exception as e:
        print("Download / Extraction of artifacts failed {}".format(str(e)))

def make_directory(directory):
    """
    Create directory

    Parameters
    ----------
    directorys : list containing the directory's path to create

    Returns
    -------
    None.

    """
    for d in directory:
        if os.path.isdir(d):
            print("directory {} already exist".format(d))
        if os.path.isdir(d)==False:
            os.mkdir(path=d)
            print("directory {} created successfully".format(d))

def delete_directory(directorys):
    """
    Delete directory

    Parameters
    ----------
    directorys : list containing the directory's path to delete along with all the files

    Returns
    -------
    None.

    """
    if len(directorys)>=1:
        for d in directorys:
            if os.path.isdir(d):
                try:
                    if os.path.isfile(d):
                        os.remove(path=d)
                    else:
                        shutil.rmtree(path=d)
                        print("Removed: {}".format(d))
                except:
                    print("Failed to removed: {}".format(d))
            else:
                print("Failed to removed: {}".format(d))

def make_archive(base_name,root_dir,zip_format='zip'):
    """
    Creates zip for given folder

    Parameters
    ----------
    base_name : name of zip file
    root_dir : directory to archive/zip
    zip_format : zip or tar
        DESCRIPTION. The default is 'zip'.

    Returns
    -------
    None.

    """
    shutil.make_archive(base_name=base_name, format=zip_format, root_dir=root_dir)

def create_folders():
    """
        Descrption: this will remove(if present previously) and create folders needed
                    to store the data , model and label for ease access

        input_parameters: None
        return_parameters: returns the path of the data , model , label , report, zip directory
    """

    data_path=os.path.join(os.getcwd(),"data")
    model_path=os.path.join(os.getcwd(),"model")

    report_path=os.path.join(os.getcwd(),"reports")
    sample_data = os.path.join(report_path, "sample_data")
    #Create Zip Path which contains data , model and label zip files
    zip_path=os.path.join(os.getcwd(),"zip")

    #deleting previously generated folders
    delete_directory(directorys=[data_path,model_path,report_path,zip_path, sample_data])

    #creating folders
    make_directory([data_path,model_path,report_path,zip_path, sample_data])

    return data_path, model_path,report_path,zip_path, sample_data


if __name__ == "__main__":

    import tqdm
    from time import sleep

    user_workflow = {
                 '1' : 'Download the Dataset',
                 '2' : 'Model Development',
                 '3' : 'Preparing Data, Model as per AIShield pre-requisite',
                 '4' : 'AIShield API Call'

           }
    if run_code:
        steps_tqdm = tqdm.tqdm(range(1,len(user_workflow)+1))
        data_path, model_path, report_path, zip_path, sample_data = create_folders()

        for step in steps_tqdm:

            if step == 1 :
                steps_tqdm.set_description(user_workflow[str(step)])
                print("Downloading of the Data Started")
                data_url = "https://aisdocs.blob.core.windows.net/reference/upload/Text/TextClassification/IMDB_Dataset.zip"
                download_extract_artifacts(data_url, os.path.basename(data_url), os.path.basename(data_path))
                print("Data downloaded succesfully")


            elif step == 2:
                steps_tqdm.set_description(user_workflow[str(step)])
                print("Downloading of the Model file Started")
                model_url = "https://aisdocs.blob.core.windows.net/reference/upload/Text/TextClassification/nlp_model.zip"
                download_extract_artifacts(model_url, os.path.basename(model_url), os.path.basename(model_path))
                print("Model file downloaded succesfully")


            elif step == 3:
                steps_tqdm.set_description(user_workflow[str(step)])

                #Zip Data
                make_archive(base_name=os.path.join(zip_path,"data"),root_dir=data_path,zip_format='zip')

                #Zip Model
                make_archive(base_name=os.path.join(zip_path,"model"),root_dir=model_path,zip_format='zip')

            elif step == 4:
                steps_tqdm.set_description(user_workflow[str(step)])
                print("Below Cells will see How to integrate and Call AIShield for Vulnerability Analysis")
                break

            sleep(1)



Started of installing all the prerequisites packages
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 20.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 66.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
librosa 0.10.1 requires numpy!=1.22.0,!=1.22.1,!=1.22.2,>=1.20.3, but you have numpy 1.22.0 which is 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.3.2
    Uninstalling charset-normalizer-3.3.2:
      Successfully uninstalled charset-normalizer-3.3.2
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.28.0 which is incompatible.
tensorflow 2.15.0 requires numpy<2.0.0,>=1.23.

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(
Download the Dataset:   0%|          | 0/4 [00:00<?, ?it/s]

Failed to removed: /content/data
Failed to removed: /content/model
Failed to removed: /content/reports
Failed to removed: /content/zip
Failed to removed: /content/reports/sample_data
directory /content/data created successfully
directory /content/model created successfully
directory /content/reports created successfully
directory /content/zip created successfully
directory /content/reports/sample_data created successfully
Data downloaded succesfully


Model Development:  25%|██▌       | 1/4 [00:04<00:14,  4.71s/it]   

Model file downloaded succesfully


AIShield API Call:  75%|███████▌  | 3/4 [00:17<00:05,  5.79s/it]

Below Cells will see How to integrate and Call AIShield for Vulnerability Analysis


<img src="https://aisdocs.blob.core.windows.net/reference/Workflow Images/wk2.png"/>

## 3.0 Vulnerability Analysis by AIShield

**STEP 5 :You can move to trigger the AIShield analysis by following steps given below. If you want to preview the generated artifact by vulnerability analysis like the report and dashboard, please find below a few samples of same, all without having to trigger an actual analysis.**

By examining these Reports, we can gain a comprehensive understanding of the upcoming Vulnerability Analysis conducted by AIShield, which will enable us to prepare more effectively.


<img src="https://aisdocs.blob.core.windows.net/reference/dashboard_images/Text_classification_MEA_Dashboard.png"/>

<img src="https://aisdocs.blob.core.windows.net/reference/dashboard_images/TC_MEA_Dashboard.png"/>

In [2]:
hosted_pdf_url = "https://aisdocs.blob.core.windows.net/reference/Reports/Text/TextClassification/MEA/VulnerabilityReport.pdf"
display_pdf( output_filename = "vul_sample",runtime_environment = runtime_environment,pdf_path =hosted_pdf_url)


#### Let's proceed with the step-by-step process of calling and integrating AIShield for Vulnerability Analysis:

1. **Initialize AIShield:** Ensure that AIShield is properly installed and its dependencies are set up. Also, initialize AIShield to prepare for the analysis.

2. **Model Registration:** Register the specific task pair (e.g. image_classification) and analysis type (e.g. Extraction / MEA) to perform the vulnerability analysis accurately.

3. **Upload Artifacts:** Upload the necessary artifacts that have been prepared previously in STEP 4 for analysis. These may include datasets, trained models, or any relevant files.

4. **Model Analysis:** Trigger the model analysis API to initiate the vulnerability analysis assessment. AIShield will perform the analysis using the uploaded artifacts.

5. **Monitor Analysis Status:** Keep track of the analysis progress and patiently wait for AIShield to generate the vulnerability assessment results. This may take some time depending on the complexity of the analysis.

6. **Download Reports & Artifacts:** Once the analysis is complete, access and download the vulnerability analysis reports and any additional artifacts generated by AIShield. Analyze these reports to gain insights into potential vulnerabilities in your system or application.

By following instructions, you can effectively call and integrate AIShield for Vulnerability Analysis.

<img src="https://aisdocs.blob.core.windows.net/reference/Workflow Images/aw.png"/>

### 3.1 Initialize AIShield

In [ ]:
"""
Description : AIShield URL , subscription key and orgid
              Initialize the
"""
base_url = "https://api.aws.boschaishield.com/prod"
url=base_url+"/api/ais/v1.5"
org_id = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' #<<Copy Org_id mentioned in welcome mail after AIShield Subscription>>


"""
Description: Initialize the AIShield API
"""


#if using AIShield API then , need an x-api-key for accessing AIShield registration / Analysis APIs
# below function will get insight of how to generate the x-api-key
def get_aws_api_key(url, org_id):

    """
    Description: to get the x_api_key
    """
    url = url+"/get_aws_api_key"
    headers = {'org_id': org_id}
    payload = {}

    x_api_key_request = requests.request("GET", url, headers=headers, data=payload)
    status_code = x_api_key_request.status_code
    x_api_key_request = json.loads(x_api_key_request.text)


    if status_code == 200:
        x_api_key = x_api_key_request['x_api_key']
        return x_api_key

    else:
        print(x_api_key_request)
        return None

#generate the x-api-key
x_api_key = get_aws_api_key(url, org_id)
print("x_api_key is :",x_api_key)

headers={'Cache-Control': 'no-cache',
'x-api-key': x_api_key,
'Org-Id' : org_id
}

'\nDescription : AIShield URL , subscription key and orgid\n              Initialize the\n'

<img src="https://aisdocs.blob.core.windows.net/reference/Workflow Images/aw1.png"/>

### 3.2 Model Registration

In [ ]:
"""
Description: Define the task and analysis type for model registration
              "task_type" : refers to the specific type of task being performed, for eg : "nlp."
              "analysis_type" : refers to the specific type of analysis being performed,for eg : "extraction.
              For more information, check out [https://docs.boschaishield.com/api-docs/lesspostgreater-model-registration#-a-xGGsdVJB3d09cBAck-]
"""

"""
Description: call Model registration api to get unique model it and url to upload data, model and label
"""
model_registration_url = url + "/model_registration/upload"
model_registration_payload = {
    'task_type':"nlp",
    "analysis_type": "mea"
}

new_request = requests.request(method="POST", url=model_registration_url, headers=headers, json=model_registration_payload)
status_cd = new_request.status_code
new_request = json.loads(new_request.text)
try:
    if status_cd == 200:
        model_id = new_request['data']['model_id']
        data_upload_obj = new_request['data']['urls']['data_upload_url']
        model_upload_obj = new_request['data']['urls']['model_upload_url']
        print('model_id: ', model_id)
    else:
        print(new_request)

except Exception as e:
    print(new_request, str(e))

'\nDescription: Define the task and analysis type for model registration\n              "task_type" : refers to the specific type of task being performed, for eg : "nlp."\n              "analysis_type" : refers to the specific type of analysis being performed,for eg : "extraction.\n              For more information, check out [https://docs.boschaishield.com/api-docs/lesspostgreater-model-registration#-a-xGGsdVJB3d09cBAck-]\n'

<img src="https://aisdocs.blob.core.windows.net/reference/Workflow Images/aw2.png"/>

### 3.3 Upload input artifacts

In [ ]:
"""
Description: Full File paths and upload input artifacts
"""
zip_path = 'zip/'
data_path=os.path.join(zip_path,'data.zip') #full path of data zip
model_path=os.path.join(zip_path,'model.zip') #full path of model zip

def upload_file(upload_obj, file_path):

    url = upload_obj['url']
    request_payload = upload_obj['fields']
    files=[
              ('file',(os.path.basename(file_path),open(file_path,'rb'),'application/zip'))
            ]

    headers = {}
    new_request = requests.request("POST", url, headers=headers, data=request_payload, files=files)
    status_cd = new_request.status_code
    if status_cd == 204:
        status = 'upload sucessful'
    else:
        status = 'upload failed'
    return status

"""
Description: Hit AIShield File Upload API
"""
data_upload_status = upload_file(data_upload_obj, data_path)
model_upload_status = upload_file(model_upload_obj, model_path)
print('data_upload_status: ', data_upload_status)
print('model_upload_status: ', model_upload_status)


'\nDescription: Full File paths and upload input artifacts\n'

<img src="https://aisdocs.blob.core.windows.net/reference/Workflow Images/aw3.png"/>

### 3.4 Model Analysis

In [ ]:
"""
Description: Specify the appropriate configs required for vulnerability analysis and trigger model analysis
"""
"""
Description: Payload for AIShield VulnerabilityReport api call
"""
payload={}
payload['use_model_api']="no"
payload['model_api_details']="no"
payload['number_of_classes']=str(num_classes)
payload['attack_type']="blackbox"
payload['number_of_attack_queries']=10000
payload['model_framework']='tensorflow'
payload['vulnerability_threshold']= 0
payload['defense_best_only']="no"
payload['encryption_strategy']= model_encryption
payload['input_dimensions']=str(input_shape)

print("nlp-Extraction parameters are:",payload)
"""
Description: Hit AIShield VulnerabilityReport api
"""
model_analysis_url = url + "/model_analyse/{}".format(model_id)
if data_upload_status == "upload sucessful" and model_upload_status == "upload sucessful":
    new_request = requests.request(method="POST", url=model_analysis_url, json=payload,headers=headers)
    new_request=json.loads(new_request.text)
    for k, v in new_request.items():
        print("* {} : {}".format(k,v))

    job_id=new_request['hashed_job_id']

'\nDescription: Payload for AIShield VulnerabilityReport api call\n'

<img src="https://aisdocs.blob.core.windows.net/reference/Workflow Images/aw4.png"/>

### 3.5 Monitor Analysis Status

In [ ]:
"""
Description: Fetch Job status using Job ID
"""

def monitor_api_progress(new_job_id):
    job_status_url = url + "/job_status_detailed?job_id=" + new_job_id

    # status dictionary
    status_dictionary = {
        'ModelExploration_Status': 'na',
        'SanityCheck_Status': 'na',
        'QueryGenerator_Status': 'na',
        'VunerabilityEngine_Status': 'na'    }
    counts = [0] * len(status_dictionary)
    failed_api_hit_count = 0
    while True:
        time.sleep(2)
        try:
            job_status_response = requests.request("GET", job_status_url, params={},
                                                   headers=headers)

            job_status_payload = json.loads(job_status_response.text)
            failing_key = 'ModelExploration_Status'
            for i, key in enumerate(status_dictionary.keys()):
                if status_dictionary[key] == 'na':
                    if job_status_payload[key] == 'inprogress' and status_dictionary[key] == 'na':
                        status_dictionary[key] = job_status_payload[key]
                        print(str(key), ":", status_dictionary[key])

                    elif job_status_payload[key] == 'completed' or job_status_payload[key] == 'passed':
                        status_dictionary[key] = job_status_payload[key]
                        counts[i] += 1
                        print(str(key), ":", status_dictionary[key])

                    if job_status_payload[key] == 'failed':
                        failing_key = key
                        status_dictionary[key] = job_status_payload[key]
                        print(str(key), ":", status_dictionary[key])

                elif job_status_payload[key] == 'completed' or job_status_payload[key] == 'passed':
                    status_dictionary[key] = job_status_payload[key]
                    if counts[i] < 1:
                        print(str(key), ":", status_dictionary[key])
                    counts[i] += 1

                else:
                    if job_status_payload[key] == 'failed':
                        failing_key = key
                        status_dictionary[key] = job_status_payload[key]
                        print(str(key), ":", status_dictionary[key])

            if job_status_payload[failing_key] == 'failed':
                break

            if status_dictionary['VunerabilityEngine_Status'] == 'passed' or status_dictionary[
                'VunerabilityEngine_Status'] == 'completed' and job_status_payload[
                'CurrentStatus'] == "Defense generation is not triggered":
                print("\n Vulnerability score {} failed to cross vulnerability threshold of {}".format(
                    job_status_payload['VulnerabiltyScore'], payload['vulnerability_threshold']))
                break

        except Exception as e:
            failed_api_hit_count += 1
            print("Error {}. trying {} ...".format(str(e), failed_api_hit_count))
            if failed_api_hit_count >= 3:
                break
    return status_dictionary

status_dictionary = monitor_api_progress(new_job_id=job_id)

'\nDescription: Fetch Job status using Job ID\n'

<img src="https://aisdocs.blob.core.windows.net/reference/Workflow Images/aw5.png"/>

### 3.6 Download Reports and artifacts

In [ ]:
report_path = "reports/"
status ="success"

In [ ]:
def download_artifact(job_id, report_path, report_type='Vulnerability', file_format=0):
    """
    job_id: job_id  received after successful api call
    report_type: report to be downloaded
    file_format: change file_format to : 0- all report in zip
                        1- report in .txt
                        2- report in .pdf
                        3- report in .json
                        4- report in .xml
    """
    print("received report_type : {} and file format is: {}".format(report_type, file_format))
    report_url = url + "/" + "get_report?job_id=" + str(
        job_id) + "&report_type=" + report_type + "&file_format=" + str(file_format)

    headers1 = headers
    headers1["content-type"] = "application/zip"

    response = requests.request("GET", report_url, params={}, headers=headers1)

    file_path = None
    if file_format == 0 or file_format == "Attack_samples":
        file_path=os.path.join(report_path, report_type + ".zip")
        with open(file_path, 'wb') as f:
            f.write(response.content)

    elif file_format == 1:
        file_path=os.path.join(report_path, report_type + ".txt")
        with open(file_path, 'wb') as f:
            f.write(response.content)

    elif file_format == 2:
        file_path=os.path.join(report_path, report_type + ".pdf")
        with open(file_path, 'wb') as f:
            f.write(response.content)

    elif file_format == 3:
        file_path=os.path.join(report_path, report_type + ".json")
        with open(file_path, 'wb') as f:
            f.write(response.content)

    elif file_format == 4:
        file_path=os.path.join(report_path, report_type + ".xml")
        with open(file_path, 'wb') as f:
            f.write(response.content)

    return file_path

In [ ]:
"""
Description: Download the Defense Reports
"""
vul_report = download_artifact(job_id=job_id, report_path= report_path, report_type='Vulnerability', file_format=2)

'\nDescription: Download the Defense Reports\n'

In [ ]:
"""
Description: Download the Attack Samples
"""
attack_report = download_artifact(job_id=job_id, report_path= report_path, report_type='Attack_samples', file_format=0)

'\nDescription: Download the Attack Samples\n'

In [ ]:

def get_file_id(report_path):

    file_id = None
    !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

    # from google.colab import drive
    # drive.mount('/content/drive')
    import os
    from googleapiclient.discovery import build
    from googleapiclient.http import MediaFileUpload

    # Authenticate and authorize access to the Google Drive API
    from google.colab import auth
    auth.authenticate_user()

    # Build the Drive API service
    drive_service = build('drive', 'v3')

    # Path to the PDF file in your local drive
#         local_pdf_path = '/content/drive/MyDrive/Vulnerability.pdf'

    # Upload the PDF file to Google Drive
    pdf_file_name = os.path.basename(report_path)
    file_metadata = {'name': pdf_file_name}
    media = MediaFileUpload(report_path, mimetype='application/pdf')
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    # Set sharing permissions to "Anyone with the link"
    file_id = file.get('id')
    drive_service.permissions().create(
        fileId=file_id,
        body={'role': 'reader', 'type': 'anyone'}
    ).execute()

    return file_id

In [ ]:
"""
Description: Displaying the current Vulnerability Assement report
"""

if ("colab" in runtime_environment.lower()):
    file_id = get_file_id(report_path = vul_report)
    display_pdf(output_filename = "vul_"+job_id[-10:],runtime_environment=runtime_environment,file_id=file_id)
else:
    display_pdf(output_filename = "vul_"+job_id[-10:],runtime_environment=runtime_environment, pdf_path =os.path.join(report_path, os.path.basename(vul_report)))

'\nDescription: Displaying the current Vulnerability Assement report\n'

<img src="https://aisdocs.blob.core.windows.net/reference/Workflow Images/aw6.png"/>